<h1><center> Welcome to MotEx </center></h1>



# First import modules, set seed parameters and import functions

In [34]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import time, random
from scipy.optimize.minpack import leastsq
from diffpy.Structure import Structure, Atom
from diffpy.srfit.pdf import PDFContribution, PDFParser, PDFGenerator
from diffpy.srfit.fitbase import FitRecipe, FitResults, Profile, FitContribution
from diffpy.srreal.pdfcalculator import DebyePDFCalculator
from multiprocessing import Pool
from functools import partial
from ase.io import read

random.seed(14)
np.random.seed(14)

In [35]:
def fitting(structure_catalogue, plot, index):
    """This function takes in a 'starting_model', and an 'index' from the 'structure_catalogue'. It generates the 
    corresponding structure and fit it to the 'Experimental_Data'."""
    
    # Read structure and divide it into two lists: Atoms we want to iterate (W) and atoms we do not iterate (O)
    stru = read(starting_model)
    xyz = stru.get_positions()
    xyz_W = xyz[:NumW].copy()
    xyz_O = xyz[NumW:len(xyz)].copy()
    keep_O = np.zeros(len(xyz_O))
    h = 0
    # Cycle through W atoms and delete W according to index 0's from permutation
    permutations = np.asarray(structure_catalogue)[:,1:]
    for j in range(len(xyz_W)):
        if permutations[index][j] == 0:
            xyz_W = np.delete(xyz_W,j - h,0)
            h = h+1   
    # Cycle through all atoms that is not iteratable and test if it is within the threshold distance. Delete atoms with no bonds
    for j in range(len(xyz_O)):        
        for k in range(len(xyz_W)):
            dist = np.linalg.norm(xyz_W[k] - xyz_O[j])
            if dist < threshold:    
                keep_O[j] = 1
                break
    h = 0            
    for j in range(len(xyz_O)):
        if keep_O[j] == 0:
            xyz_O = np.delete(xyz_O,j - h, 0)
            h += 1
            
    # Create structure for iterable (W) and non-iterable (O) atoms and combine them
    W_cluster = Structure([Atom('Fe', xi) for xi in xyz_W])
    O_cluster = Structure([Atom('O', xi) for xi in xyz_O])
    cluster = W_cluster + O_cluster
    
    # Make a standard cluster refinement using Diffpy-CMI
    # Import the data and make it a PDFprofile. Define the range of the data that will be used in the fit.
    pdfprofile = Profile()
    pdfparser = PDFParser()
    pdfparser.parseFile(Experimental_Data)
    pdfprofile.loadParsedData(pdfparser)
    pdfprofile.setCalculationRange(xmin = 1, xmax = 30)

    # Setup the PDFgenerator that calculates the PDF from the structure
    pdfgenerator_cluster = PDFGenerator("G")
    # Add the profile and both generators to the PDFcontribution
    pdfcontribution = FitContribution("pdf")
    pdfcontribution.setProfile(pdfprofile, xname="r") 
    pdfcontribution.addProfileGenerator(pdfgenerator_cluster)
    
    pdfgenerator_cluster.setQmin(1)
    pdfgenerator_cluster.setQmax(17)
    pdfgenerator_cluster._calc.evaluatortype = 'OPTIMIZED'
    pdfgenerator_cluster.setStructure(cluster, periodic = False)

    # Use scaling factors proportional to molar content
    pdfcontribution.setEquation('mc*G')

    # Define the recipe to do the fit and add it to the PDFcontribution
    recipe = FitRecipe()
    recipe.addContribution(pdfcontribution)

    # Avoid too much output during fitting 
    recipe.clearFitHooks()

    # Add the scale factor.
    recipe.addVar(pdfcontribution.mc, 1.0, tag = "scale")
    
    # Add the instrumental parameters to the two generators
    pdfgenerator_cluster.qdamp.value = 0.04
    pdfgenerator_cluster.qbroad.value = 0.00

    # Add the delta2 parameters, and make sure it cannot take unphysical values
    recipe.addVar(pdfgenerator_cluster.delta2, 0, name = "delta2_cluster", tag = "delta2")

    # Add ADP and "cell" for the cluster
    phase_cluster = pdfgenerator_cluster.phase
    atoms = phase_cluster.getScatterers()
    lat = phase_cluster.getLattice()

    recipe.newVar("zoomscale1", 1.0, tag = "lat")
    recipe.newVar("zoomscale2", 1.0, tag = "lat")
    recipe.newVar("zoomscale3", 1.0, tag = "lat")
    recipe.constrain(lat.a, 'zoomscale1')
    recipe.constrain(lat.b, 'zoomscale2')
    recipe.constrain(lat.c, 'zoomscale3')

    W_cluster = recipe.newVar("W_Biso_cluster1", 0.4, tag = 'adp_w')
    O_cluster = recipe.newVar("O_Biso_cluster1", 0.4, tag = 'adp_o')

    for atom in atoms:
        if atom.element.title() == "Fe":
            recipe.constrain(atom.Biso, W_cluster)
        elif atom.element.title() == "O":
            recipe.constrain(atom.Biso, O_cluster)

    recipe.restrain("zoomscale1", lb = 0.99, ub = 1.01, sig = 0.001)
    recipe.restrain("zoomscale2", lb = 0.99, ub = 1.01, sig = 0.001)
    recipe.restrain("zoomscale3", lb = 0.99, ub = 1.01, sig = 0.001)
    
    #free parameters are set
    recipe.fix('all')
    recipe.free("scale", "lat")

    # Turn off printout of iteration number.
    #recipe.clearFitHooks()

    # We can now execute the fit using scipy's least square optimizer.
    leastsq(recipe.residual, recipe.getValues())
    
    # We calculate the goodness-of-fit, Rwp
    g = recipe.pdf.profile.y
    gcalc = recipe.pdf.evaluate()
    rfactor1 = np.sqrt(sum((g - gcalc)**2) / sum((g)**2))
    
    # if plot == 1 it will also plot the fit
    if plot == 1:
        print ("FIT RESULTS\n")
        res1 = FitResults(recipe)
        print (res1)

        # Plot the observed and refined PDF.
        # Get the experimental data from the recipe
        r = recipe.pdf.profile.x
        gobs = recipe.pdf.profile.y

        # Get the calculated PDF and compute the difference between the calculated and measured PDF
        gcalc = recipe.pdf.evaluate()
        baseline = 1.1 * gobs.min()
        gdiff = gobs - gcalc

        # Plot!
        plt.figure()
        plt.plot(r, gobs, 'bo', label="G(r) data")
        plt.plot(r, gcalc, 'r-', label="G(r) fit")
        plt.plot(r, gdiff + baseline, 'g-', label="G(r) diff")
        plt.plot(r, np.zeros_like(r) + baseline, 'k:')
        plt.xlabel(r"$r (\AA)$")
        plt.ylabel(r"$G (\AA^{-2})$")
        plt.legend()

        plt.show()
    return rfactor1

def fitting_multiprocess(structure_catalogue, SaveName, cores=1):
    """This function runs the refinement of all the structures in the structure catalogue using multiprocessing"""
    start_time = time.time()
    values = []
    # Set up multiprocessing refinement
    fitindex = range(len(structure_catalogue))
    p = Pool(processes=cores)
    plot = 0
    func = partial(fitting, structure_catalogue, plot)
    results = p.map(func, fitindex)
    p.close()
    p.join()
    
    # Start refinement and append results to lists
    for i in fitindex:
        if i % 100 == 0:
            print ("I have now fitted: ", str(i) + " structures out of " + str(len(structure_catalogue)))
        rw = results[i]
        values.append(i)
        values.append(rw)
    values = np.reshape(values,(int(len(values)/2) , 2))
    
    # Save results in format that is suitable for Machine Learning
    print ("Best fit")
    print (values[np.argmin(values[:,1])])
    print("Total execution time: %.3fs" % (time.time()-start_time))
    Result = np.column_stack([values, np.asarray(structure_catalogue)[values[:,0].astype(int)]])
    np.savetxt(SaveName, Result)
    return Result

def structure_catalogue_maker(Number_of_atoms):
    structure_catalogue = []
    for i in range(Number_of_atoms):
        my_list = np.ones((Number_of_atoms+1,))
        my_list[0] = Number_of_atoms-1
        my_list[i+1] = 0
        my_list = list(my_list)
        structure_catalogue.append(my_list)
    print ("Permutations Succeeded")
    return structure_catalogue

def calculate_atomContributionValue(Result, saveResults, id_number):
    """Calculate atom contribution value list from the result array"""
    AtomContributionValues = []
    for i in range(len(Result)):
        AtomContributionValues.append(1-Result[i,1])
    np.savetxt(saveResults + "/AtomContributionValues.txt", AtomContributionValues)
    
    Norm_AtomContributionValues = AtomContributionValues.copy()
    # Normalise the AtomContributionValues and get the RGB color in viridis.reverse
    amin, amax = min(Norm_AtomContributionValues), max(Norm_AtomContributionValues)
    for i, val in enumerate(Norm_AtomContributionValues):
        Norm_AtomContributionValues[i] = (val-amin) / (amax-amin)
    Norm_AtomContributionValues_ph = Norm_AtomContributionValues.copy()
    Norm_AtomContributionValues_ph.sort()
    
    # Normalise such that a threshold is set on the the 10 % lowest and 10 % highest atoms
    norm = mpl.colors.Normalize(vmin=Norm_AtomContributionValues_ph[round((len(Norm_AtomContributionValues))/10)], vmax=Norm_AtomContributionValues_ph[-round((len(Norm_AtomContributionValues))/10)])
    cmap = mpl.cm.cividis_r
    
    print ("Atom contribution are calculated to: ")
    for i in range(len(AtomContributionValues)):
        m = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
        print ("Atom #", str(i+1) + ": ", str(AtomContributionValues[i]), " Colorcode: ", mpl.colors.rgb2hex(m.to_rgba(Norm_AtomContributionValues[i])))
    
    f = open(saveResults+"AtomContributionValues"+str(id_number)+".txt", "w")
    f.write("\nAtom contribution are calculated to: \n")
    for i in range(len(AtomContributionValues)):
        m = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
        f.write("Atom # "+ str(i+1) + ":  "+ str(AtomContributionValues[i]) + "  Colorcode:  "+ mpl.colors.rgb2hex(m.to_rgba(Norm_AtomContributionValues[i]))+"\n")
    
    f.close()
    
    return AtomContributionValues


# Define variables

In [36]:
starting_model = "Structure_Models/35643-ICSD_222unitcell.xyz" # Name of the starting model file
Number_of_structures = 5000 # Number of structures made to the structure catalogue
NumW = 200 # Number of atoms that should be permuted in the starting model
threshold = 2.5 # Thredshold for W - O bond
frames = 151
saveResults = "Results/"


# Run

In [4]:
for frame in range(frames):
    
    # Step 1: Make the structure catalogue
    structure_catalogue = structure_catalogue_maker(Number_of_atoms=NumW)
    
    # Step 2: Fit all of the structures from the catalogue of structure motifs to the dataset
    ### First define the experimental data path and the path you want the structure catalogue with fits to be saved
    Experimental_Data = "Experimental_Data/Fe2O3_3204M_"+str(frame)+".gr" # Name of the experimental file
    saveFits = "Training_Data/Fe2O3_3204M_"+str(frame)+".txt" # Name of the saved fits file

    ### Produce organized structure catalogue with Rwp values
    Result = fitting_multiprocess(structure_catalogue, SaveName=saveFits, cores=None)
    
    # Step 4: Calculate Atom Contribution values
    calculate_atomContributionValue(Result, saveResults, id_number=frame)
    
    
    # Output a crystalmaker file
    colors = open(saveResults+"AtomContributionValues"+str(frame)+".txt", "r")

    XYZ = open(starting_model, "r")
    CrystalMaker = open(saveResults+'CrystalMaker_'+str(frame)+'.cmtx', 'w')
    lines_XYZ = XYZ.readlines()
    lines_colors = colors.readlines()

    CrystalMaker.write("MOLE  CrystalMaker molecule format\n")
    CrystalMaker.write("TITL  Molecule\n\n")
    CrystalMaker.write("! Model type\n")
    CrystalMaker.write("MODL  1\n\n")

    CrystalMaker.write("! Depth fading settings\n")
    CrystalMaker.write("DCUE  1.000000 0.212899 0.704686\n\n")

    CrystalMaker.write("! Colour definitions:\n")
    CrystalMaker.write("TYPE\n")

    for iter, line in enumerate(lines_XYZ[2:]):
        if iter + 2 < len(lines_colors):
            CrystalMaker.write(line[:line.find(" ")] + str(iter+1) + " 1.32 ")
            rgb1 = mpl.colors.to_rgb(lines_colors[iter+2].split(" ")[-1].replace("\n", ""))[0]
            rgb2 = mpl.colors.to_rgb(lines_colors[iter+2].split(" ")[-1].replace("\n", ""))[1]
            rgb3 = mpl.colors.to_rgb(lines_colors[iter+2].split(" ")[-1].replace("\n", ""))[2]
            CrystalMaker.write(str(rgb1) + " " + str(rgb2) + " " + str(rgb3))
            CrystalMaker.write("\n")
        else:
            CrystalMaker.write(line[:line.find(" ")] + str(iter+1) + " 0.66 ")
            rgb1 = mpl.colors.to_rgb("#FF0000")[0]
            rgb2 = mpl.colors.to_rgb("#FF0000")[1]
            rgb3 = mpl.colors.to_rgb("#FF0000")[2]
            CrystalMaker.write(str(rgb1) + " " + str(rgb2) + " " + str(rgb3))
            CrystalMaker.write("\n")
    CrystalMaker.write("\n")
    CrystalMaker.write("! Atoms list\n")
    CrystalMaker.write("! Bond Specifications\n")
    for iter, line in enumerate(lines_XYZ[2:]):
        if iter + 2 < len(lines_colors):
            CrystalMaker.write("BMAX " + line[:line.find(" ")] + str(iter+1) + " O  2.6")
            CrystalMaker.write("\n")


    CrystalMaker.write("\n")
    CrystalMaker.write("! Atoms list\n")
    CrystalMaker.write("ATOM\n")

    for iter, line in enumerate(lines_XYZ[2:]):
        if iter + 2 < len(lines_colors):
            CrystalMaker.write(line[:line.find(" ")] + str(iter+1) + " Fe" + str(iter+1) + " " + line.split(" ")[1:][0] + " " + line.split(" ")[1:][1] + " " + line.split(" ")[1:][2])
        else:
            CrystalMaker.write(line[:line.find(" ")] + " O" + str(iter+1) + " " + line.split(" ")[1:][0] + " " + line.split(" ")[1:][1] + " " + line.split(" ")[1:][2])

    CrystalMaker.close()



Permutations Succeeded
I have now fitted:  0 structures out of 200
I have now fitted:  100 structures out of 200
Best fit
[153.           0.98878269]
Total execution time: 1794.020s
Atom contribution are calculated to: 
Atom # 1:  0.010663046751423377  Colorcode:  #cdbb63
Atom # 2:  0.010744686608805343  Colorcode:  #a39a74
Atom # 3:  0.010609841006862464  Colorcode:  #ead34c
Atom # 4:  0.010719347396728529  Colorcode:  #afa471
Atom # 5:  0.01063351452364436  Colorcode:  #ddc858
Atom # 6:  0.010738331146591062  Colorcode:  #a69c74
Atom # 7:  0.010588362068734258  Colorcode:  #f7de3e
Atom # 8:  0.010740790978554293  Colorcode:  #a59c74
Atom # 9:  0.011030810758537624  Colorcode:  #083370
Atom # 10:  0.01088733613914783  Colorcode:  #62656f
Atom # 11:  0.011035500459221415  Colorcode:  #013271
Atom # 12:  0.011078756115146304  Colorcode:  #00234f
Atom # 13:  0.01116765102519135  Colorcode:  #00224e
Atom # 14:  0.010921071473789401  Colorcode:  #535a6d
Atom # 15:  0.01121497415319428  Col

Atom # 191:  0.010591788801191937  Colorcode:  #f5dc41
Atom # 192:  0.010677181727426488  Colorcode:  #c5b568
Atom # 193:  0.010640300551042814  Colorcode:  #dac65b
Atom # 194:  0.010639533909765131  Colorcode:  #dac65b
Atom # 195:  0.010677876610527104  Colorcode:  #c5b568
Atom # 196:  0.010589541869708063  Colorcode:  #f6dd3f
Atom # 197:  0.010691704449320438  Colorcode:  #beaf6b
Atom # 198:  0.01073586679793992  Colorcode:  #a79d73
Atom # 199:  0.010564572379767156  Colorcode:  #fee838
Atom # 200:  0.010659414393747069  Colorcode:  #cfbd62
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
1

Atom # 196:  0.01011742643639435  Colorcode:  #f6dd3f
Atom # 197:  0.010216452002443366  Colorcode:  #bdae6c
Atom # 198:  0.0102477651408438  Colorcode:  #aca172
Atom # 199:  0.01009632371364333  Colorcode:  #fee838
Atom # 200:  0.010181931374079856  Colorcode:  #d0be62
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209

3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278


29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296

...

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454


Process ForkPoolWorker-123:
Traceback (most recent call last):
  File "/Users/andysodeanker/opt/anaconda3/envs/py37/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/andysodeanker/opt/anaconda3/envs/py37/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/andysodeanker/opt/anaconda3/envs/py37/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/Users/andysodeanker/opt/anaconda3/envs/py37/lib/python3.7/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "<ipython-input-2-cededbcd810d>", line 127, in fitting
    leastsq(recipe.residual, recipe.getValues())
  File "/Users/andysodeanker/opt/anaconda3/envs/py37/lib/python3.7/site-packages/scipy/optimize/minpack.py", line 424, in leastsq
    gtol, maxfev, epsfcn, factor, diag)
  File "/Users/andysodeanker/opt/anaconda3/envs/py37/lib/python3.7/s

  File "/Users/andysodeanker/opt/anaconda3/envs/py37/lib/python3.7/site-packages/diffpy.srfit-3.0.0-py3.7.egg/diffpy/srfit/equation/equationmod.py", line 202, in __call__
    self._value = self.root.getValue()
  File "/Users/andysodeanker/opt/anaconda3/envs/py37/lib/python3.7/site-packages/diffpy.srfit-3.0.0-py3.7.egg/diffpy/srfit/equation/literals/operators.py", line 118, in getValue
    vals = [l.value for l in self.args]
  File "/Users/andysodeanker/opt/anaconda3/envs/py37/lib/python3.7/site-packages/diffpy.srfit-3.0.0-py3.7.egg/diffpy/srfit/equation/literals/operators.py", line 118, in <listcomp>
    vals = [l.value for l in self.args]
  File "/Users/andysodeanker/opt/anaconda3/envs/py37/lib/python3.7/site-packages/diffpy.srfit-3.0.0-py3.7.egg/diffpy/srfit/equation/literals/operators.py", line 122, in <lambda>
    value = property(lambda self: self.getValue())
  File "/Users/andysodeanker/opt/anaconda3/envs/py37/lib/python3.7/site-packages/diffpy.srfit-3.0.0-py3.7.egg/diffpy/srfit/

KeyboardInterrupt: 

# Test by renormalising

In [39]:
import re
saveResults = "Results/"
frames = 151
min_AtomContributionValue = 1
max_AtomContributionValue = 0
for frame in range(30):
    # Output a crystalmaker file
    colors = open(saveResults+"AtomContributionValues"+str(frame)+".txt", "r")
    for line in colors.readlines():
        for ph in line.split(" "):
            AtomContributionValue = re.findall("\d+\.\d+", ph)
            if len(AtomContributionValue) > 0:
                AtomContributionValue = float(AtomContributionValue[0])
                if AtomContributionValue < min_AtomContributionValue:
                    min_AtomContributionValue = AtomContributionValue
                elif AtomContributionValue > max_AtomContributionValue:
                    max_AtomContributionValue = AtomContributionValue
print ("min_AtomContributionValue: ", min_AtomContributionValue)
print ("max_AtomContributionValue: ", max_AtomContributionValue)

for frame in range(30):
    AtomContributionValues = []
    # Output a crystalmaker file
    colors = open(saveResults+"AtomContributionValues"+str(frame)+".txt", "r")
    for line in colors.readlines():
        for ph in line.split(" "):
            AtomContributionValue = re.findall("\d+\.\d+", ph)
            if len(AtomContributionValue) > 0:
                AtomContributionValue = float(AtomContributionValue[0])
                AtomContributionValues.append(AtomContributionValue)
                                              
    Norm_AtomContributionValues = AtomContributionValues.copy()
    # Normalise the AtomContributionValues and get the RGB color in viridis.reverse
    amin, amax = min_AtomContributionValue, max_AtomContributionValue
    for i, val in enumerate(Norm_AtomContributionValues):
        Norm_AtomContributionValues[i] = (val-amin) / (amax-amin)
    Norm_AtomContributionValues_ph = Norm_AtomContributionValues.copy()
    Norm_AtomContributionValues_ph.sort()
    
    # Normalise such that a threshold is set on the the 10 % lowest and 10 % highest atoms
    norm = mpl.colors.Normalize(vmin=Norm_AtomContributionValues_ph[round((len(Norm_AtomContributionValues))/10)], vmax=Norm_AtomContributionValues_ph[-round((len(Norm_AtomContributionValues))/10)])
    cmap = mpl.cm.cividis_r
    
    print ("Atom contribution are calculated to: ")
    for i in range(len(AtomContributionValues)):
        m = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
        print ("Atom #", str(i+1) + ": ", str(AtomContributionValues[i]), " Colorcode: ", mpl.colors.rgb2hex(m.to_rgba(Norm_AtomContributionValues[i])))
    
    f = open(saveResults+"AtomContributionValues_norm"+str(frame)+".txt", "w")
    f.write("\nAtom contribution are calculated to: \n")
    for i in range(len(AtomContributionValues)):
        m = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
        f.write("Atom # "+ str(i+1) + ":  "+ str(AtomContributionValues[i]) + "  Colorcode:  "+ mpl.colors.rgb2hex(m.to_rgba(Norm_AtomContributionValues[i]))+"\n")
    
    f.close()
    
    # Output a crystalmaker file
    colors = open(saveResults+"AtomContributionValues_norm"+str(frame)+".txt", "r")

    XYZ = open(starting_model, "r")
    CrystalMaker = open(saveResults+'CrystalMaker_norm_'+str(frame)+'.cmtx', 'w')
    lines_XYZ = XYZ.readlines()
    lines_colors = colors.readlines()

    CrystalMaker.write("MOLE  CrystalMaker molecule format\n")
    CrystalMaker.write("TITL  Molecule\n\n")
    CrystalMaker.write("! Model type\n")
    CrystalMaker.write("MODL  1\n\n")

    CrystalMaker.write("! Depth fading settings\n")
    CrystalMaker.write("DCUE  1.000000 0.212899 0.704686\n\n")

    CrystalMaker.write("! Colour definitions:\n")
    CrystalMaker.write("TYPE\n")

    for iter, line in enumerate(lines_XYZ[2:]):
        if iter + 2 < len(lines_colors):
            CrystalMaker.write(line[:line.find(" ")] + str(iter+1) + " 1.32 ")
            rgb1 = mpl.colors.to_rgb(lines_colors[iter+2].split(" ")[-1].replace("\n", ""))[0]
            rgb2 = mpl.colors.to_rgb(lines_colors[iter+2].split(" ")[-1].replace("\n", ""))[1]
            rgb3 = mpl.colors.to_rgb(lines_colors[iter+2].split(" ")[-1].replace("\n", ""))[2]
            CrystalMaker.write(str(rgb1) + " " + str(rgb2) + " " + str(rgb3))
            CrystalMaker.write("\n")
        else:
            CrystalMaker.write(line[:line.find(" ")] + str(iter+1) + " 0.66 ")
            rgb1 = mpl.colors.to_rgb("#FF0000")[0]
            rgb2 = mpl.colors.to_rgb("#FF0000")[1]
            rgb3 = mpl.colors.to_rgb("#FF0000")[2]
            CrystalMaker.write(str(rgb1) + " " + str(rgb2) + " " + str(rgb3))
            CrystalMaker.write("\n")
    CrystalMaker.write("\n")
    CrystalMaker.write("! Atoms list\n")
    CrystalMaker.write("! Bond Specifications\n")
    for iter, line in enumerate(lines_XYZ[2:]):
        if iter + 2 < len(lines_colors):
            CrystalMaker.write("BMAX " + line[:line.find(" ")] + str(iter+1) + " O  2.6")
            CrystalMaker.write("\n")


    CrystalMaker.write("\n")
    CrystalMaker.write("! Atoms list\n")
    CrystalMaker.write("ATOM\n")

    for iter, line in enumerate(lines_XYZ[2:]):
        if iter + 2 < len(lines_colors):
            CrystalMaker.write(line[:line.find(" ")] + str(iter+1) + " Fe" + str(iter+1) + " " + line.split(" ")[1:][0] + " " + line.split(" ")[1:][1] + " " + line.split(" ")[1:][2])
        else:
            CrystalMaker.write(line[:line.find(" ")] + " O" + str(iter+1) + " " + line.split(" ")[1:][0] + " " + line.split(" ")[1:][1] + " " + line.split(" ")[1:][2])

    CrystalMaker.close()

min_AtomContributionValue:  0.004212845463190007
max_AtomContributionValue:  0.048711060667237294
Atom contribution are calculated to: 
Atom # 1:  0.010663046751423377  Colorcode:  #cdbb63
Atom # 2:  0.010744686608805343  Colorcode:  #a39a74
Atom # 3:  0.010609841006862464  Colorcode:  #ead34c
Atom # 4:  0.010719347396728529  Colorcode:  #afa471
Atom # 5:  0.01063351452364436  Colorcode:  #ddc858
Atom # 6:  0.010738331146591062  Colorcode:  #a69c74
Atom # 7:  0.010588362068734258  Colorcode:  #f7de3e
Atom # 8:  0.010740790978554293  Colorcode:  #a59c74
Atom # 9:  0.011030810758537624  Colorcode:  #083370
Atom # 10:  0.01088733613914783  Colorcode:  #62656f
Atom # 11:  0.011035500459221415  Colorcode:  #013271
Atom # 12:  0.011078756115146304  Colorcode:  #00234f
Atom # 13:  0.01116765102519135  Colorcode:  #00224e
Atom # 14:  0.010921071473789401  Colorcode:  #535a6d
Atom # 15:  0.01121497415319428  Colorcode:  #00224e
Atom # 16:  0.010850047924222284  Colorcode:  #727274
Atom # 17:  0

Atom # 67:  0.010096986753840298  Colorcode:  #fee838
Atom # 68:  0.01023354744418803  Colorcode:  #b4a76f
Atom # 69:  0.010170727837701898  Colorcode:  #d6c35d
Atom # 70:  0.010184747236202774  Colorcode:  #cebc63
Atom # 71:  0.010181968894842242  Colorcode:  #d0be62
Atom # 72:  0.010127066458519685  Colorcode:  #efd748
Atom # 73:  0.010162515113447124  Colorcode:  #dbc75a
Atom # 74:  0.010049805895418484  Colorcode:  #fee838
Atom # 75:  0.010146719676252025  Colorcode:  #e4ce53
Atom # 76:  0.01015780407285194  Colorcode:  #ddc858
Atom # 77:  0.010256376617145957  Colorcode:  #a79d73
Atom # 78:  0.010199577143123362  Colorcode:  #c6b667
Atom # 79:  0.010119297634619828  Colorcode:  #f5dc41
Atom # 80:  0.01012213700067044  Colorcode:  #f2da44
Atom # 81:  0.010473825072017773  Colorcode:  #3e4b6c
Atom # 82:  0.010535478545184884  Colorcode:  #0f3570
Atom # 83:  0.010517674518084608  Colorcode:  #213b6e
Atom # 84:  0.010510609776432034  Colorcode:  #273e6e
Atom # 85:  0.01059809881200268

Atom # 162:  0.00826907069737215  Colorcode:  #fee838
Atom # 163:  0.008335742132805968  Colorcode:  #e8d24f
Atom # 164:  0.008346827040767946  Colorcode:  #e1cc55
Atom # 165:  0.008400001056779405  Colorcode:  #c1b26a
Atom # 166:  0.008325361159170708  Colorcode:  #efd748
Atom # 167:  0.008362925851939318  Colorcode:  #d7c45c
Atom # 168:  0.008360122318319507  Colorcode:  #d9c55c
Atom # 169:  0.008394923486634509  Colorcode:  #c3b369
Atom # 170:  0.008318250179130127  Colorcode:  #f3db42
Atom # 171:  0.008369197402969486  Colorcode:  #d3c05f
Atom # 172:  0.008343346656720985  Colorcode:  #e4ce53
Atom # 173:  0.008445981105151823  Colorcode:  #a69c74
Atom # 174:  0.008446532884014424  Colorcode:  #a59c74
Atom # 175:  0.008371954986554297  Colorcode:  #d1bf61
Atom # 176:  0.008294721053505594  Colorcode:  #fee838
Atom # 177:  0.008583168873167013  Colorcode:  #5e636f
Atom # 178:  0.008781892577108041  Colorcode:  #00224e
Atom # 179:  0.008680023688035665  Colorcode:  #243c6e
Atom # 180:

Atom # 174:  0.01027596375098061  Colorcode:  #aaa073
Atom # 175:  0.010203977950294152  Colorcode:  #d0be62
Atom # 176:  0.0101151698099744  Colorcode:  #fee838
Atom # 177:  0.01044200169213616  Colorcode:  #5e626e
Atom # 178:  0.010663944084106092  Colorcode:  #00224e
Atom # 179:  0.010553220507567018  Colorcode:  #233c6e
Atom # 180:  0.01057630291101086  Colorcode:  #083370
Atom # 181:  0.010504918803119656  Colorcode:  #404c6c
Atom # 182:  0.010719700628057938  Colorcode:  #00224e
Atom # 183:  0.010381499596886168  Colorcode:  #787877
Atom # 184:  0.01076822641371089  Colorcode:  #00224e
Atom # 185:  0.009975861179774559  Colorcode:  #fee838
Atom # 186:  0.010078523873777612  Colorcode:  #fee838
Atom # 187:  0.010168841691625574  Colorcode:  #e4ce53
Atom # 188:  0.010161977497746277  Colorcode:  #e7d150
Atom # 189:  0.01019644025820432  Colorcode:  #d4c15f
Atom # 190:  0.010167745944337891  Colorcode:  #e4ce53
Atom # 191:  0.01013010966614869  Colorcode:  #f9e03a
Atom # 192:  0.010

Atom # 174:  0.009399961983807659  Colorcode:  #a89e73
Atom # 175:  0.009326575941747794  Colorcode:  #d1bf61
Atom # 176:  0.00924179149014892  Colorcode:  #fee838
Atom # 177:  0.009557932883112574  Colorcode:  #5d616e
Atom # 178:  0.009763966341108099  Colorcode:  #00224e
Atom # 179:  0.009661218672937788  Colorcode:  #233c6e
Atom # 180:  0.009682657007042739  Colorcode:  #083370
Atom # 181:  0.00961086909672837  Colorcode:  #424e6c
Atom # 182:  0.009809848922206488  Colorcode:  #00224e
Atom # 183:  0.00949865065255584  Colorcode:  #777777
Atom # 184:  0.009856345935466582  Colorcode:  #00224e
Atom # 185:  0.009125436398332742  Colorcode:  #fee838
Atom # 186:  0.009208933199195024  Colorcode:  #fee838
Atom # 187:  0.009299562131602768  Colorcode:  #e0cb56
Atom # 188:  0.009286794639609797  Colorcode:  #e7d150
Atom # 189:  0.00932602179983022  Colorcode:  #d1bf61
Atom # 190:  0.009297532174605605  Colorcode:  #e1cc55
Atom # 191:  0.009269270852849454  Colorcode:  #f2da44
Atom # 192:  0

...

Atom contribution are calculated to: 
Atom # 1:  0.037180112482831484  Colorcode:  #d6c35d
Atom # 2:  0.03746522713273759  Colorcode:  #737475
Atom # 3:  0.03715480941036664  Colorcode:  #e0cb56
Atom # 4:  0.037409655053083024  Colorcode:  #858279
Atom # 5:  0.03712187699044178  Colorcode:  #edd54a
Atom # 6:  0.037343244881136495  Colorcode:  #9b9476
Atom # 7:  0.03710592156877934  Colorcode:  #f3db42
Atom # 8:  0.03731095839073517  Colorcode:  #a69c74
Atom # 9:  0.03767000890940364  Colorcode:  #2e416d
Atom # 10:  0.03759614016624935  Colorcode:  #4a536c
Atom # 11:  0.037780107224545945  Colorcode:  #002859
Atom # 12:  0.03786413763509622  Colorcode:  #00224e
Atom # 13:  0.03795057194199147  Colorcode:  #00224e
Atom # 14:  0.03759186642248513  Colorcode:  #4c556c
Atom # 15:  0.03795439924143362  Colorcode:  #00224e
Atom # 16:  0.037546144947555815  Colorcode:  #5b606e
Atom # 17:  0.037180112410968746  Colorcode:  #d6c35d
Atom # 18:  0.036820650577846226  Colorcode:  #fee838
Atom # 19:

Atom contribution are calculated to: 
Atom # 1:  0.038815463020861696  Colorcode:  #d1bf61
Atom # 2:  0.039113297019836235  Colorcode:  #757575
Atom # 3:  0.038783337020154796  Colorcode:  #dcc859
Atom # 4:  0.039053674613410916  Colorcode:  #858279
Atom # 5:  0.0387494791048828  Colorcode:  #e8d24f
Atom # 6:  0.03898759333948332  Colorcode:  #999277
Atom # 7:  0.03872890891609704  Colorcode:  #efd748
Atom # 8:  0.03896517222779716  Colorcode:  #a09875
Atom # 9:  0.039368433702623884  Colorcode:  #243c6e
Atom # 10:  0.03926493583638435  Colorcode:  #4a536c
Atom # 11:  0.03948687015771768  Colorcode:  #00234f
Atom # 12:  0.03957130045359669  Colorcode:  #00224e
Atom # 13:  0.03966391961863058  Colorcode:  #00224e
Atom # 14:  0.039275466124061964  Colorcode:  #47516c
Atom # 15:  0.0396984653374276  Colorcode:  #00224e
Atom # 16:  0.03920188643840383  Colorcode:  #5d616e
Atom # 17:  0.03881546302709604  Colorcode:  #d1bf61
Atom # 18:  0.03842702552920241  Colorcode:  #fee838
Atom # 19:  0

Atom contribution are calculated to: 
Atom # 1:  0.043875897846600176  Colorcode:  #dac65b
Atom # 2:  0.044234104338696234  Colorcode:  #727274
Atom # 3:  0.04383973388339091  Colorcode:  #e6d051
Atom # 4:  0.04415182387553551  Colorcode:  #878478
Atom # 5:  0.04382215070300599  Colorcode:  #ebd44b
Atom # 6:  0.04410084556694949  Colorcode:  #958f77
Atom # 7:  0.04378978113634713  Colorcode:  #f7de3e
Atom # 8:  0.04406417225150017  Colorcode:  #a09875
Atom # 9:  0.04447622285289066  Colorcode:  #2b406d
Atom # 10:  0.04439317036142354  Colorcode:  #46516c
Atom # 11:  0.04462167657530958  Colorcode:  #00224e
Atom # 12:  0.04471063425397315  Colorcode:  #00224e
Atom # 13:  0.04481281912792723  Colorcode:  #00224e
Atom # 14:  0.04440279869028374  Colorcode:  #444f6c
Atom # 15:  0.04481061921959739  Colorcode:  #00224e
Atom # 16:  0.04432825248035899  Colorcode:  #595e6e
Atom # 17:  0.04387589823633131  Colorcode:  #dac65b
Atom # 18:  0.04345933674454705  Colorcode:  #fee838
Atom # 19:  0.0

Atom # 189:  0.044026710297210725  Colorcode:  #aba072
Atom # 190:  0.04386503776402895  Colorcode:  #dec958
Atom # 191:  0.043905775974797945  Colorcode:  #d1bf61
Atom # 192:  0.04392123155335581  Colorcode:  #ccba64
Atom # 193:  0.04389799468537792  Colorcode:  #d3c05f
Atom # 194:  0.04389961083949445  Colorcode:  #d2c060
Atom # 195:  0.04405376718146592  Colorcode:  #a39a74
Atom # 196:  0.04383425088675297  Colorcode:  #e8d24f
Atom # 197:  0.043923771022212676  Colorcode:  #cbb965
Atom # 198:  0.0442829647894325  Colorcode:  #656870
Atom # 199:  0.04382613011454173  Colorcode:  #ead34c
Atom # 200:  0.043985207271476146  Colorcode:  #b8aa6e
Atom contribution are calculated to: 
Atom # 1:  0.04271129440168853  Colorcode:  #d5c25e
Atom # 2:  0.04303084221552245  Colorcode:  #777777
Atom # 3:  0.042678598002306  Colorcode:  #dfca57
Atom # 4:  0.042968489310088454  Colorcode:  #898578
Atom # 5:  0.042661908761868816  Colorcode:  #e5cf52
Atom # 6:  0.042908082559103944  Colorcode:  #99927

Atom contribution are calculated to: 
Atom # 1:  0.047640090601627105  Colorcode:  #d7c45c
Atom # 2:  0.04799267589971923  Colorcode:  #777777
Atom # 3:  0.0476176281002092  Colorcode:  #dec958
Atom # 4:  0.047921916129907816  Colorcode:  #8a8678
Atom # 5:  0.04758586970086687  Colorcode:  #e8d24f
Atom # 6:  0.047855373995058126  Colorcode:  #9b9476
Atom # 7:  0.04756611735679406  Colorcode:  #eed649
Atom # 8:  0.04782215346574148  Colorcode:  #a49b74
Atom # 9:  0.048274093580232824  Colorcode:  #2f426d
Atom # 10:  0.048179173653554286  Colorcode:  #4b546c
Atom # 11:  0.04842543781367814  Colorcode:  #002656
Atom # 12:  0.04852324414444864  Colorcode:  #00224e
Atom # 13:  0.04864900046088161  Colorcode:  #00224e
Atom # 14:  0.0481814382713982  Colorcode:  #4a536c
Atom # 15:  0.04866544763919545  Colorcode:  #00224e
Atom # 16:  0.04811713356317082  Colorcode:  #5b606e
Atom # 17:  0.04764009058670293  Colorcode:  #d7c45c
Atom # 18:  0.04723733739148761  Colorcode:  #fee838
Atom # 19:  0.